In [2]:
!pip install selenium
!pip install scrapy

In [3]:
!pip install fake_useragent

In [4]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [5]:
#title
from fake_useragent import UserAgent
ua = UserAgent()
userAgent = ua.chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f'user-agent={userAgent}')
# Create a new instance of the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Navigate to a website
url = 'https://www.imdb.com/title/tt4154796/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0'
driver.get(url)

# Get the page title
page_title = driver.title
print(f"Page Title: {page_title}")

# Close the browser
driver.quit()

Page Title: Avengers: Endgame (2019) - User reviews - IMDb


In [39]:
#Reviews in page 1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# IMDb URL
url = 'https://www.imdb.com/title/tt4154796/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0'
driver.get(url)

# Wait for the reviews to load
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'lister-list')))

# Extract and print all review texts
review_elements = driver.find_elements(By.CLASS_NAME, 'text.show-more__control')
for i, review_element in enumerate(review_elements, start=1):
    print(f"Review {i}: {review_element.text.strip()}")

# Close the WebDriver
driver.quit()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001028d2654 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x00000001028ca8c8 cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x000000010241e714 cxxbridge1$string$len + 90428
3   chromedriver                        0x00000001024657c0 cxxbridge1$string$len + 381416
4   chromedriver                        0x00000001024a6de8 cxxbridge1$string$len + 649232
5   chromedriver                        0x00000001024599c8 cxxbridge1$string$len + 332784
6   chromedriver                        0x0000000102896278 cxxbridge1$str$ptr + 2476360
7   chromedriver                        0x000000010289950c cxxbridge1$str$ptr + 2489308
8   chromedriver                        0x0000000102877a64 cxxbridge1$str$ptr + 2351412
9   chromedriver                        0x0000000102899d94 cxxbridge1$str$ptr + 2491492
10  chromedriver                        0x0000000102868d58 cxxbridge1$str

In [ ]:
#all reviews and ratings
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# IMDb URL for Avengers: Endgame reviews
url = 'https://www.imdb.com/title/tt4154796/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0'
driver.get(url)

# Wait for the reviews to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'lister-list')))

# Function to click "Load More" button until all reviews are loaded
def load_all_reviews():
    while True:
        try:
            load_more_button = driver.find_element(By.CLASS_NAME, 'ipl-load-more__button')
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, 'ipl-load-more__load-indicator')))
        except Exception as e:
            break

# Function to extract ratings and reviews
def extract_data():
    rating_elements = driver.find_elements(By.CLASS_NAME, 'ipl-ratings-bar')
    review_elements = driver.find_elements(By.CLASS_NAME, 'text')

    data = []
    for rating_element, review_element in zip(rating_elements, review_elements):
        rating = rating_element.text.strip().split('\n')[0]
        review = review_element.text.strip()
        data.append({'Rating': rating if rating else None, 'Review': review if review else None})
        print(f"Rating: {rating if rating else 'None'}")
        print(f"Review: {review if review else 'None'}\n")
    return data

# Load all reviews and extract the ratings and reviews
load_all_reviews()
all_data = extract_data()

# Close the WebDriver
driver.quit()

In [ ]:
# Extract text content from review elements
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Create a DataFrame to store the ratings and reviews
ratings_reviews_df = pd.DataFrame(all_data)

# Save DataFrame to Excel with ratings and reviews in different columns
output_file = "/content/drive/MyDrive/Reviews.xlsx"
ratings_reviews_df.to_excel(output_file, index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the Excel file into a DataFrame
ratings_reviews_df = pd.read_excel("/content/drive/MyDrive/Reviews.xlsx")

# Group the data by the "ratings" column and count occurrences
ratings_counts = ratings_reviews_df['Rating'].value_counts()

# Sort the ratings_counts Series in descending order
ratings_counts = ratings_counts.sort_values(ascending=False)

# Define colors for each rating value
colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan', 'brown','pink','navy','yellow']

# Plot a bar graph
ax = ratings_counts.plot(kind='bar', color=colors)
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Add number counts on each bar
for i, count in enumerate(ratings_counts):
    ax.text(i, count + 10, str(count), ha='center', va='bottom')

plt.show()



SENTIMENT ANALYSIS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
import pandas as pd

# Load the data from the Excel file
input_file = "/content/drive/MyDrive/Reviews.xlsx"
ratings_reviews_df = pd.read_excel(input_file)

# Function to calculate sentiment score based on ratings
def calculate_sentiment(rating):
    # Extract the numerical part of the rating (e.g., "7/10" -> 7)
    rating_value = float(rating.split('/')[0])
    # Calculate sentiment score
    sentiment_score = 1 if rating_value > 7 else 0
    return sentiment_score

# Apply the calculate_sentiment function to the 'Rating' column and add the result as a new column 'Sentiment'
ratings_reviews_df['Sentiment'] = ratings_reviews_df['Rating'].apply(calculate_sentiment)

# Save the updated DataFrame to Excel with the sentiment score added
output_file = "/content/drive/MyDrive/Reviews-1S.xlsx"
ratings_reviews_df.to_excel(output_file, index=False)

# Print the updated DataFrame
print(ratings_reviews_df)

In [ ]:
import pandas as pd

training = pd.read_excel("/content/drive/MyDrive/Reviews-1S.xlsx", header=0)
print(training.head())

In [ ]:
y_train = training['Sentiment']
x_train = training.drop(["Sentiment","Rating"], axis=1)

print(x_train.head())
print(y_train.head())

In [ ]:
y_train.value_counts()

CLEANING - DATA PREPROCESSING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#read file with reviews
import pandas as pd

# Read the Excel file into a DataFrame

df = pd.read_excel('/content/drive/MyDrive/Reviews-1S.xlsx')
print(df)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##### Training Data
import pandas as pd
training= pd.read_excel('/content/drive/MyDrive/Reviews-1S.xlsx')
print(training.shape)
training.head()

In [ ]:
# Data pre-processing

import nltk
import numpy as np
import matplotlib.pyplot as plt

# Import list of stopwords from library NLTK
from nltk.corpus import stopwords
nltk.download('stopwords')


In [ ]:
# Remove Stopwords

stopwords_list = stopwords.words("english")
print(f'List of stopwords:\n{stopwords_list}\n')

# We remove negation words in list of stopwords
no_stopwords = ["not","don't",'aren','don','ain',"aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
               'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't",
               "won't", 'wouldn', "wouldn't"]
for no_stopword in no_stopwords:
    stopwords_list.remove(no_stopword)

print(f'Final list of stopwords:\n{stopwords_list}')

In [ ]:
# Lemmatize reviews

# Import Lemmatizer from NLTK
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# function that receive a list of words and do lemmatization:
def lemma_stem_text(words_list):
    # Lemmatizer
    text = [lemmatizer.lemmatize(token.lower()) for token in words_list] # eighties->eight or messages->message or drugs->drug
    text = [lemmatizer.lemmatize(token.lower(), "v") for token in text ]# going-> go or started->start or watching->watch
    return text

In [ ]:
#Negative Contractions

# create a function to change negative contractions (e.g. isn't, can't, ... etc.) to standard from using a regular expression.
import re
re_negation = re.compile("n't ") # specify a pattern you want to find in a string

# function that receive a sequence of words and return the same sequence transforming
# abbreviated negations to the standard form.
def negation_abbreviated_to_standard(sent):
    sent = re_negation.sub(" not ", sent)
    return sent


In [ ]:
#REVIEW

# create a function to clean the text of a review using the functions defined previously.

# Import function BeautifulSoup to clean text of HTML tags
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from collections import Counter
nltk.download('punkt')

def review_to_words(raw_review):
    # 1. Remove HTML tags
    review_text = BeautifulSoup(raw_review).get_text()

    # 2. Transform abbreviated negations to the standard form.
    review_text = negation_abbreviated_to_standard(review_text)

    # 3. Remove non-letters and non-numbers
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(review_text)

    # 4. Remove stop words
    meaningful_words = [w for w in words if w.lower() not in stopwords_list]

    # 5. Apply lemmatization function
    lemma_words = lemma_stem_text(meaningful_words)

    # 6. Join the words back into one string separated by space, and return the result.
    return( " ".join(lemma_words))

    # 7. Handling Short Words
    tokens = [word for word in tokens if len(word) >= 3]

    # 8. Handling Rare Words
    word_counts = Counter(tokens)
    common_words = set(word for word, count in word_counts.items() if count > 5)
    tokens = [word for word in tokens if word in common_words]

    # 9. # Drop rows with NaN values in the 'Reviews' column
    training.dropna(subset=['Review'], inplace=True)

In [ ]:
# Clean the first review
clean_review = review_to_words(training["Review"].iloc[0])

# Print original review, and cleaned review
print(f'Text of original review:\n{training["Review"].iloc[0]}\n')
print(f'Text of cleaned review:\n{clean_review}')

In [ ]:
# We get the text of reviews in the training set
reviews = x_train['Review']

# We initialize an empty list to add the clean reviews
cleaned_train_reviews = []

# We loop over each review and clean it
for i in reviews:
    cleaned_train_reviews.append(review_to_words(i))


In [ ]:
# Print ALL cleaned reviews
print(cleaned_train_reviews)

In [ ]:
#wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all cleaned reviews into a single string
all_reviews_text = ' '.join(cleaned_train_reviews)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_reviews_text)

# Display the word cloud
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

DOCUMENT TERM MATRIX- (DTM)

Logistic

In [ ]:
# Import Count Vectorizer encoding from sklearn library
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
c = CountVectorizer(stop_words='english', lowercase=True, token_pattern=r'\w+', ngram_range=(1, 2))

# Fit and transform y_train to generate the Bigram DTM
train_data_features_d = c.fit_transform(cleaned_train_reviews)

In [ ]:
# We split train dataset to create validation set and training the Logistic Regression Model on validation set
from sklearn.model_selection import train_test_split
X_train_d, X_val_d, Y_train_d, Y_val_d = train_test_split(train_data_features_d, y_train, test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize a logistic regression model
logistic_d = LogisticRegression(random_state=0)
# Train the model
logistic_d.fit(X_train_d, Y_train_d)


# Predict on the validation set
y_pred = logistic_d.predict(X_val_d)


In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
Y_pred_logistic_d  = logistic_d.predict(X_val_d)
confusion_matrix_logistic_d = confusion_matrix(Y_val_d, Y_pred_logistic_d, labels=[1,0])
confusion_matrix_logistic_d

In [ ]:
# plot the confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
sns.heatmap(confusion_matrix_logistic_d, annot=True, fmt="d")
ax.set_title('Confusion matrix Logistic Regression')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score,roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score


# Calculate accuracy and round to 2 decimal places
accuracy = round(accuracy_score(Y_val_d, y_pred), 2)
print("Accuracy:", accuracy)

# Predict on the validation set
y_pred_d = logistic_d.predict(X_val_d)

# Calculate precision
precision_d = precision_score(Y_val_d, y_pred_d)

# Calculate recall
recall_d = recall_score(Y_val_d, y_pred_d)

# Calculate F1 score
f1_d = f1_score(Y_val_d, y_pred_d)


print("Precision (DTM):", precision_d)
print("Recall (DTM):", recall_d)
print("F1 Score (DTM):", f1_d)


SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Initialize SVM model
svm_model = SVC(random_state=0)

#Train the SVM model
svm_model.fit(X_train_d, Y_train_d)

#Predict on the validation set
Y_pred_svm_d = svm_model.predict(X_val_d)



In [ ]:
# Calculate accuracy for SVM model
accuracy_svm = accuracy_score(Y_val_d, Y_pred_svm_d)
precision_svm = precision_score(Y_val_d, Y_pred_svm_d)
recall_svm = recall_score(Y_val_d, Y_pred_svm_d)
f1_svm = f1_score(Y_val_d, Y_pred_svm_d)

print("\nSVM Model Metrics:")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1 Score:", f1_svm)

Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize Random Forest model
random_forest_model = RandomForestClassifier(random_state=0)

# Train the Random Forest model
random_forest_model.fit(X_train_d, Y_train_d)

# Predict on the validation set
Y_pred_rf_d = random_forest_model.predict(X_val_d)

# Calculate metrics for Random Forest model
accuracy_rf = accuracy_score(Y_val_d, Y_pred_rf_d)
precision_rf = precision_score(Y_val_d, Y_pred_rf_d)
recall_rf = recall_score(Y_val_d, Y_pred_rf_d)
f1_rf = f1_score(Y_val_d, Y_pred_rf_d)

print("Random Forest Model Metrics:")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1 Score:", f1_rf)


TF-IDF

In [ ]:
# Import tf-idf encoding from sklearn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Define some hyperparameters of encoded
vectorizer = TfidfVectorizer(min_df=0.5, max_df=0.90,ngram_range = (1,2))

# Create the training set with the words encoded as features of the reviews
train_data_features_t = vectorizer.fit_transform(cleaned_train_reviews)

In [ ]:
# We split train dataset to create validation set and training the Logistic Regression Model on validation set
from sklearn.model_selection import train_test_split
X_train_t, X_val_t, Y_train_t, Y_val_t = train_test_split(train_data_features_t, y_train, test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize a logistic regression model
logistic_t = LogisticRegression(random_state=0)
# Train the model
logistic_t = logistic_t.fit(X_train_t, Y_train_t)


In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
Y_pred_logistic_t  = logistic_t.predict(X_val_t)
confusion_matrix_logistic_t = confusion_matrix(Y_val_t, Y_pred_logistic_t, labels=[1,0])
confusion_matrix_logistic_t

In [ ]:
# plot the confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
sns.heatmap(confusion_matrix_logistic_t, annot=True, fmt="d")
ax.set_title('Confusion matrix Logistic Regression')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the validation set
y_pred_t = logistic_t.predict(X_val_t)

# Calculate metrics for logistic regression model
accuracy_logistic_t = accuracy_score(Y_val_t, y_pred_t)
precision_logistic_t = precision_score(Y_val_t, y_pred_t)
recall_logistic_t = recall_score(Y_val_t, y_pred_t)
f1_logistic_t = f1_score(Y_val_t, y_pred_t)

print("Logistic Regression Model Metrics on TF-IDF encoded data:")
print("Accuracy:", accuracy_logistic_t)
print("Precision:", precision_logistic_t)
print("Recall:", recall_logistic_t)
print("F1 Score:", f1_logistic_t)


SVM

In [ ]:
from sklearn.svm import SVC

# Initialize SVM model
svm_model_tfidf = SVC(random_state=0)

# Train the SVM model
svm_model_tfidf.fit(X_train_t, Y_train_t)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the validation set
y_pred_svm_tfidf = svm_model_tfidf.predict(X_val_t)

# Calculate metrics for SVM model
accuracy_svm_tfidf = accuracy_score(Y_val_t, y_pred_svm_tfidf)
precision_svm_tfidf = precision_score(Y_val_t, y_pred_svm_tfidf)
recall_svm_tfidf = recall_score(Y_val_t, y_pred_svm_tfidf)
f1_svm_tfidf = f1_score(Y_val_t, y_pred_svm_tfidf)

print("SVM Model Metrics on TF-IDF encoded data:")
print("Accuracy:", accuracy_svm_tfidf)
print("Precision:", precision_svm_tfidf)
print("Recall:", recall_svm_tfidf)
print("F1 Score:", f1_svm_tfidf)


Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=0)

# Train the Random Forest model
rf_model.fit(X_train_t, Y_train_t)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the validation set
y_pred_rf_t = rf_model.predict(X_val_t)

# Calculate metrics for Random Forest model
accuracy_rf_t = accuracy_score(Y_val_t, y_pred_rf_t)
precision_rf_t = precision_score(Y_val_t, y_pred_rf_t)
recall_rf_t = recall_score(Y_val_t, y_pred_rf_t)
f1_rf_t = f1_score(Y_val_t, y_pred_rf_t)

print("Random Forest Model Metrics:")
print("Accuracy:", accuracy_rf_t)
print("Precision:", precision_rf_t)
print("Recall:", recall_rf_t)
print("F1 Score:", f1_rf_t)


WORD EMBEDDINGS - (word2vec)

In [ ]:
pip install gensim

In [ ]:
import gensim

# Tokenize the cleaned reviews
tokenized_reviews = [review.split() for review in cleaned_train_reviews]

# Train the Word2Vec model
model = gensim.models.Word2Vec(
    tokenized_reviews,
    vector_size=150,
    window=10,
    min_count=3,
    workers=10
)

# Train the model for 10 epochs
model.train(tokenized_reviews, total_examples=len(tokenized_reviews), epochs=10)

# Save the trained word vectors
word_vectors = model.wv
word_vectors.save("/content/drive/MyDrive/STAT653/my_w2v.kv")

# Load the saved word vectors
loaded_word_vectors = gensim.models.KeyedVectors.load("/content/drive/MyDrive/STAT653/my_w2v.kv")


In [ ]:
w1 = "thanos"
loaded_word_vectors.most_similar (positive=w1,topn=5)

In [ ]:
w2 = "love"
loaded_word_vectors.most_similar (positive=w2, topn=4)

In [ ]:
from gensim.models import Word2Vec

# Tokenize the cleaned reviews
tokenized_reviews = [review.split() for review in cleaned_train_reviews]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Save the Word2Vec model
word2vec_model.save("/content/drive/MyDrive/word2vec_model.model")

word2vec_model = Word2Vec.load("/content/drive/MyDrive/word2vec_model.model")


In [ ]:
similar_words = word2vec_model.wv.most_similar('great', topn=5)
print(similar_words)

In [ ]:
similar_words = word2vec_model.wv.most_similar('thanos', topn=5)
print(similar_words)

In [ ]:
w1 = "love"
similar_words = word2vec_model.wv.most_similar(positive=w1, topn=3)
print(similar_words)

In [ ]:
similarity_score = word2vec_model.wv.similarity(w1="love", w2="3000")

# Print the similarity score
print(similarity_score)

In [ ]:
# Find the odd word out
odd_one_out = word2vec_model.wv.doesnt_match(["thor", "thanos", "captain"])

# Print the odd word out
print(odd_one_out)

In [ ]:
# Find the odd word out
odd_one_out = word2vec_model.wv.doesnt_match(["film", "thor", "ironman"])

# Print the odd word out
print(odd_one_out)

FASTTEXT

In [ ]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import FastText
import pandas as pd

tokenized_reviews = [review.split() for review in cleaned_train_reviews]
phrases = Phrases(tokenized_reviews, min_count = 30, progress_per = 10000)
sentences = phrases[tokenized_reviews]

In [ ]:
# Initialize the model
model = FastText(vector_size=100, window=5, min_count=1, workers=4)

# Building Vocabulary
model.build_vocab(sentences)

# Get the keys using the .index_to_key method
keys = model.wv.index_to_key

print(len(keys))

In [ ]:
#Training the model
model.train(sentences, total_examples=model.corpus_count, epochs=10)

# Save the trained model using joblib
import joblib
joblib.dump(model, '/content/drive/MyDrive/STAT653/fasttext_model.joblib')

In [ ]:
vocabulary = model.wv.index_to_key
print('uniform' in vocabulary)

In [ ]:
vocabulary = model.wv.index_to_key
print('3000' in vocabulary)

In [ ]:
from gensim.models import FastText

# Tokenize your reviews data
tokenized_reviews = [review.split() for review in cleaned_train_reviews]

# Train the FastText model
model = FastText(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained model
model.save("/content/drive/MyDrive/fasttext_model")

In [ ]:
# Calculate similarity between two words
similarity = model.wv.similarity("great", "amazing")
print("Similarity between 'great' and 'amazing':", similarity)

In [ ]:
# Calculate similarity between two words
similarity = model.wv.similarity("action", "adventure")
print("Similarity between 'action' and 'adventure':", similarity)

In [ ]:
!pip install fasttext
import fasttext

In [ ]:
# Define the path for the output file
output_file = '/content/drive/MyDrive/cleaned_train_reviews.txt'

# Write the cleaned reviews to the output file
with open(output_file, 'w') as file:
    file.write('\n'.join(cleaned_train_reviews))

print("Cleaned train reviews have been saved in:", output_file)


In [ ]:
import fasttext
import time
start_time = time.time()
path = '/content/drive/MyDrive/cleaned_train_reviews.txt'
model = fasttext.train_unsupervised(path , model='skipgram')
end_time = time.time()
print(f"training time took {round(end_time - start_time, 3)} secs")
print(model.words)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([model['performance']], [model['beautiful']]))